##Requirements
Before running this notebook make sure you have the following:
- An Azure ML workspace
- A compute instance

# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice
import requests
import json

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
#Retrieve current workspace
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Tuscan_Wine_Forecast'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')



## Compute Cluster
If compute cluster already exists, it will just identify it and end.


In [ ]:
#Create Cluster if does not already exist

cluster_name = "myCluster"
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Cluster already created")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS12_V2",min_nodes=1, max_nodes=6)
    cluster = ComputeTarget.create(ws,cluster_name, compute_config) #creates the actual cluster

cluster.wait_for_completion(show_output=True)  #Allows to continue on other threads while cluster is being created


#SOURCE / HELP: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute.amlcompute?view=azure-ml-py

## Dataset
A Dataset is created from the csv file in the folder "data".

In [ ]:
# Create a dataset from delimited files with header option as ONLY_FIRST_FILE_HAS_HEADERS

   datastore = ws.get_default_datastore() 
   dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'data/dataset_Tuscany_Wine_prepared.csv'),
       header='ONLY_FIRST_FILE_HAS_HEADERS')

   df = dataset.to_pandas_dataframe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
#automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data= dataset,
    validation_size = 0.33,
    label_column_name='points',
    compute_target = cluster
    )

In [ ]:
# TODO: Submit your experiment
runAutoML = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(runAutoML).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = runAutoML.get_output()
print(best_run)
         
print_model(fitted_model)

RunDetails(best_run).show()

In [ ]:
#TODO: Save the best model

# Register run as Model

model_name = "AleaumeModelAutoML"
description = "Best AutoML Model"
model_path ="outputs/modelAutoML.pkl"

#model = best_run.register_model(model_name = model_name, description = description, model_path = model_path )

model = runAutoML.register_model(model_name = model_name, description = description)

#SOURCE / HELP: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python

#Unfortunately so far unsuccessful to register the model via SDK and get a default generated swagger.json uri. Also,  MSFT support recommends to do so via AML studio...

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#To create a local deployment configuration
deployment_config = LocalWebservice.deploy_configuration(port=9064)

#Create Environment

env = Environment(name="AzureML-Tutorial")
myenv=env.clone("myenv")
myenv.python.conda_dependencies.add_pip_package("joblib==1.1.0")

my_inference_config = InferenceConfig(
    environment=env,
    source_directory="./",
    entry_script="./score_dummy.py"
    #entry_script="./score.py"
)

# Deploy the service locally

service = model.deploy(ws, "local-service", [model], my_inference_config, deployment_config)
service.reload()
print(service.get_logs())

print(service.scoring_uri)

service.wait_for_deployment(show_output=True)


TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
#Call model to test 

service.update(enable_app_insights=True)


uri = service.scoring_uri
requests.get("http://localhost:9064")
headers = {"Content-Type": "application/json"}
data = {
    "query": "This is a test query"
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

# Deleting the webservice


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
